In [1]:
import requests
import lxml
import bs4
from bs4 import BeautifulSoup
import re
import pandas as pd

In [2]:
# Cities with >100k population
cities_page = 'https://wikitravel.org/en/Wikitravel:World_cities'

In [3]:
page = requests.get(cities_page)
soup = BeautifulSoup(page.text, 'html5lib')
body = soup.find('body')

In [4]:
countries = []
cities = []
cities_URL = []

curr_country = 'Afghanistan' # Initialize w/ Afgh.
prev_title = 'Afghanistan' # Initialize w/ Afgh.
counter = 0
rows_to_del = []

# Loop through links
for item in body.find_all('a', href=True): 
    if '#' in str(item): # Skip Table of Contents
        continue
    if "does not exist" in str(item): # Skip pages that do not exist
        continue
    if "external autonumber" in str(item): # Skip other irrelevant links
        continue
    if "Mutare" in str(item): # Stop after final city
        break 
        
    # If a new country was started
    if "Edit section" in str(item):
        curr_country = prev_title # Switch to new country
        rows_to_del.append(counter-1)
        continue # and don't append this line
    
    # Save current title
    prev_title = item['title']
    
    # Append data to list
    cities_URL.append([counter,curr_country,(item['title']),('wikitravel.org'+item['href'])])
    counter+=1

In [5]:
# Convert to pandas df
cities_URL_df_raw = pd.DataFrame(cities_URL)
cities_URL_df = cities_URL_df_raw.drop(rows_to_del)

In [6]:
cities_URL_df.reset_index(inplace=True)

In [7]:
cities_URL_df = cities_URL_df.iloc[:,2:5]

In [8]:
cities_URL_df.head()

,1,2,3
0,Afghanistan,Herat,wikitravel.org/en/Herat
1,Afghanistan,Kabul,wikitravel.org/en/Kabul
2,Afghanistan,Jalalabad,wikitravel.org/en/Jalalabad
3,Afghanistan,Kandahar,wikitravel.org/en/Kandahar
4,Afghanistan,Kunduz,wikitravel.org/en/Kunduz


In [9]:
cities_URL_df.to_csv('wikitravel_city_URLs.csv')